In [3]:
from urllib.request import urlopen
import pandas as pd
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import os

os.chdir(r'C:\Users\borde\Desktop\COVID\Risk Locator')
Rates_pd = pd.read_excel('Output.xlsx',index_col=False)
#Rates_pd = Rates_pd.drop(['index'], axis=1)
Rates_pd = Rates_pd[Rates_pd['FIPS']>1000]
Rates_pd = Rates_pd[Rates_pd['FIPS']<60000].reset_index()
TrackedFIPS_pd = pd.DataFrame(list(zip(Rates_pd['FIPS'], Rates_pd['Incidence_Rate'], Rates_pd['Active'])), columns = ['FIPS', 'Incidence_Rate', 'Active'])

os.chdir(r'C:\Users\borde\Desktop\COVID')
FIPS_pd = pd.read_csv('FIPS Lookup.csv',index_col=False)
FIPS_pd = FIPS_pd.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','Lat', 'Long_', 'Population'],axis=1).dropna(subset=['FIPS'])
FIPS_pd = FIPS_pd[FIPS_pd['FIPS']>1000]
FIPS_pd = FIPS_pd[FIPS_pd['FIPS']<60000].reset_index()
AllFIPS_pd = pd.DataFrame(list(zip(FIPS_pd['FIPS'])), columns = ['FIPS'])

missing_pd = pd.merge(AllFIPS_pd, TrackedFIPS_pd, how='outer', indicator=True)

locations = FIPS_pd['Combined_Key']
locations = [x[:-4] for x in locations]

missing_pd['Location'] = locations
missing_pd['FIPS'] = missing_pd['FIPS'].astype('int64', copy=True)

risk = []

missing_pd['Incidence_Rate'] = missing_pd['Incidence_Rate'].fillna(0)
missing_pd['Active'] = missing_pd['Active'].fillna(0)

std = Rates_pd['Incidence_Rate'].std()
mean = Rates_pd['Incidence_Rate'].mean()


for x in range(0,len(missing_pd)):
    if missing_pd['Incidence_Rate'][x]<=mean + std:
        risk.append('LOW')
    elif missing_pd['Incidence_Rate'][x]>mean + std and missing_pd['Incidence_Rate'][x]<=mean+2*std:
        risk.append('MEDIUM')
    elif missing_pd['Incidence_Rate'][x]>mean+2*std:
        risk.append('HIGH')
    else: 
        risk.append('ERROR')
        
missing_pd['Risk'] = risk

strFIPS = []

for x in range(0,len(missing_pd)):
    strFIPS.append(str(missing_pd['FIPS'][x]).rjust(5,'0'))

missing_pd['str_FIPS'] = strFIPS

import plotly.express as px

colorscale = ["#CCFFCC","#00FF00","#99FF00","#CCFF00","#FFFF00","#FFCC00","#FF6600","#FF0000"]

fig = px.choropleth(missing_pd, geojson=counties, locations='str_FIPS', color='Incidence_Rate',
                    color_continuous_scale=colorscale,
                    range_color=(0, mean+std*2), 
                    scope="usa", title='Active Incidence Rate per County', hover_name = "Location", hover_data=['Risk', 'Active'])

fig.update_traces(marker_line_width=.3, marker_opacity=.8,hovertemplate='<b>%{hovertext}</b><br>Risk: %{customdata[0]}<br><br>Incidence Rate: %{z}<br>Active Cases: %{customdata[1]}<extra></extra>')
fig.update_layout(legend = dict(x=0.8),title_x = 0.4, font={"size":20, "color":"white"},geo=dict(bgcolor='#323130', lakecolor='#323130', subunitcolor='black'), plot_bgcolor='#111110', paper_bgcolor='#111110',margin={"r":0,"t":100,"l":0,"b":50})
fig.update_geos(showsubunits=True, subunitcolor='black')
print('working..')
os.chdir(r'C:\Users\borde\Documents\Incidence Map')
fig.write_html('index2.html',auto_open=True)
print('done')

working..
done
